# Генерация праздничных тостов 
## Выбор предобученной модели
Для работы с GPT скачаем предобученную модель.
Для дообучения модели под генерацию русскоязычных праздничных тостов была взята предобученная модель от сбера `RuGPT-3Medium`. Конечно, взяв модель побольше `RuGPT-3Large`, результат генерации был бы лучше, но получить доступ к суперкомпьютеру, способному справиться с Large-моделью, не удалось.

In [ ]:
!pip install transformers 

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)


## Обучающие данные
Для получения датасета спарсим сайт https://pozdravok.com/ и выведем полученные данные в dataset.txt.

In [ ]:
from bs4 import BeautifulSoup
import requests
import re

f=open('dataset.txt', 'a',encoding='utf-8')

for page in range(2, 100):
    url = "https://pozdravok.com/pozdravleniya/a/proza-" + str(page) + ".htm"

    get_url = requests.get(url)

    soup = BeautifulSoup(get_url.text, "html.parser")
    teme = soup.find('div', class_ = "p_descr")

    for element in soup.find('div', class_ = "content"):
        f.write(str(element.text).replace(' ',' ') + '\n')

f.close()

Также перед загрузкой данных необходимо добавить символы [ST] и [ET] - символы начала и конца тоста.

In [ ]:
with open("dataset.txt", 'r', encoding='utf-8') as file:
    f = open('dataset_with_ST_ET.txt', 'w', encoding='utf-8')
    contents = file.readlines()
    
    for i in range(0,len(contents)):
        string_dataset = str(contents[i])
        string_dataset = string_dataset.replace('\n','')

        if string_dataset!='':
            f.write('[ST]' + string_dataset + '[ET]' + '\n\n\n\n')


Теперь загрузим и обработаем полученный датасет

In [ ]:
!pip install datasets
from datasets import load_dataset, Dataset
dataset = load_dataset("text", data_files={"train": "dataset_with_ST_ET.txt"}, sample_by="document")

In [16]:
toast_datasets = []
for item in dataset['train'][0]['text'].split('\n\n\n\n'):
    toast_datasets.append({"text":item})

dataset = Dataset.from_list(toast_datasets, split="train")
dataset

Dataset({
    features: ['text'],
    num_rows: 1330
})

In [17]:
def tokenize_text(examples):
    return tokenizer(examples["text"], truncation=True, max_length=128)
dataset = dataset.map(tokenize_text, batched=True, remove_columns=["text"])  ##токенизируем полученный обработанный датасет

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

## Обучение
Зададим параметры обучения

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned", 
    overwrite_output_dir=True,
    num_train_epochs=25,
    logging_steps=250,
    save_steps=1000,
    per_device_train_batch_size=2,  #размер пакета (количества данных) на ядро для обучения
    per_device_eval_batch_size=2,  #размер пакета (количества данных) на ядро для оценки
    warmup_steps=10,  #снижаем скорость обучения, чтобы уменьшить влияние отклонения модели от обучения
    gradient_accumulation_steps=16,  ##количество шагов обновления для накопления градиента (чтобы увеличить размер пакета)
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)  #указание оптимизатора со скоростью обучения 1e-5
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#!g1.1
trainer.train()

***** Running training *****
  Num examples = 1330
  Num Epochs = 25
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 16
  Total optimization steps = 1025
  Number of trainable parameters = 355871744


Step,Training Loss
250,0.075900
500,0.077200
750,0.080200
1000,0.088400


Saving model checkpoint to ./finetuned/checkpoint-1000
Configuration saved in ./finetuned/checkpoint-1000/config.json
Model weights saved in ./finetuned/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1025, training_loss=0.0808626821564465, metrics={'train_runtime': 2186.737, 'train_samples_per_second': 15.205, 'train_steps_per_second': 0.469, 'total_flos': 3857822685069312.0, 'train_loss': 0.0808626821564465, 'epoch': 24.99})

# Результат обучения
Сохранение модели и токенайзера

In [ ]:
model.save_pretrained("model_best")
tokenizer.save_pretrained("model_best")

Для генерации праздичного тоста будем использовать сэмплирование с ограничением маловероятных токенов, чтобы обеспечить генерацию, проходящую случайным образом, и при этом исключить всякий бред

In [ ]:
model.eval()

text = "С Новым Годом!\n"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5,  #количество лучей генерации
                        temperature=1.8,  #параметр температуры, определяющий степень рандомности (чем больше температура, тем больше степень рандомности)
                        top_p=0.8,  #ограничение, отсекающее маловероятные токены (сумма вероятностей сета токенов должна быть не больше p)
                        top_k=50,  #ограничение, отсекающее маловероятные токены (зануляются все вероятности кроме k самых вероятных)
                        max_length=120, 
                        min_length=30,
                        no_repeat_ngram_size=2
                        )

generated_text = list(map(tokenizer.decode, out))[0]
out_text = generated_text.replace('[SJ]', '').split('[ET]')[0]

print()
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


С Новым Годом!

Пусть этот новый год будет полон счастливых мгновений, улыбок и веселья! Пусть сбываются самые заветные желания, а близкие будут здоровы и благополучны! Желаю, чтобы Дедушка Мороз не поскупился на щедрые подарки, и чтобы волшебная ночь началась с приятных хлопушек и веселого гвалта. Пусть этот Новый год станет ярким, незабываемым и принесет множество приятных сюрпризов! С праздником, с Новым годом!
